<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/preprocessing_pdfs_chunk_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openparse[ml] html2text langchain langchain_groq langchain_huggingface faiss-cpu langchain-pinecone pinecone-notebooks
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc_2018.pdf"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 8.3 MB/s eta 0:00:00
--2024-09-08 23:09:54--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1383621 (1.3M) [application/pdf]
Saving to: ‘data/ppc_2018.pdf’

data/ppc_2018.pdf   100%[===================>]   1.32M   996KB/s    in 1.4s    

2024-09-08 23:09:57 (996 KB/s) - ‘data/ppc_2018.pdf’ saved [1383621/1383621]



In [2]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class MetadadosChunk(BaseModel):
    topico: str = Field(description="Identifique do que se trata o texto: ementa de XXX, grade curricular , regulamento XXX, regras, laboratório, objetivos, infraestrutura, corpo docente, etc. Exemplos: ementa de Grafos, Instrução de validação de disciplinas, etc")
    palavras_chave: str = Field(description="Cinco palavras-chave principais que representam os conceitos centrais do chunk, separadas por vírgula.")
    possiveis_perguntas: str = Field(description="Lista de possíveis perguntas que podem ser respondidas com base no conteúdo deste chunk, separadas por vírgula")


In [85]:
from langchain_groq import ChatGroq

def get_metadata(chunk, model_id='llama3-groq-70b-8192-tool-use-preview', count=0):
    try:
        model = ChatGroq(model=model_id, api_key=userdata.get('GROQ_API_KEY'))

        prompt = f"""
        Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS.
        Sua função é fornecer metadados sobre textos extraídos do PPC.
        Os tópicos comumente são sobre: grades curriculares, ementas, regulamentos, regras, objetivos, infraestrutura, corpo docente e assuntos relacionados.
        Analise o seguinte texto e forneça os seguintes metadados:
        - Tópico
        - Palavras-chave
        - Possiveis perguntas
        Texto :  {chunk}
        """

        structured_llm = model.with_structured_output(MetadadosChunk)
        completion = structured_llm.invoke(prompt)
        return completion

    except Exception as e:
        if count < 3:
            print(f"Erro ao tentar invocar o modelo: {e}. Tentativa {count + 1} de 3.")
            return get_metadata(chunk, model_id, count + 1)

        print("Falha em todas as tentativas. Retornando metadados padrão.")
        return MetadadosChunk(
            topico="Tópico desconhecido",
            palavras_chave="Desconhecido",
            possiveis_perguntas="Desconhecida",
        )


## Tables

We aim to be model agnostic - the DocumentParser supports extracting tables using either the "table-transformers" or "pymupdf" libraries - we're model agnostic. The `parsing_algorithm` field in the configuration dictionary decides which one to use.

The `PyMuPDFArgsDict` (and similarly the `TableTransformersArgsDict`) lets you fine-tune how tables are extracted using specific arguments.

In [5]:
import openparse
import html2text
from langchain_core.documents import Document
from uuid import uuid4


def parse_document_to_documents(file_path, source_name):
    # Parse the document using OpenParse
    parser = openparse.DocumentParser(table_args={"parsing_algorithm": "pymupdf", "table_output_format": "markdown"})
    document = parser.parse(file_path)

    documents = []
    ids = []

    for node in document.nodes:
        html_content = node.text

        if html_content:
            id = str(uuid4())
            ids.append(id)
            document_chunk = Document(
                metadata={'source': source_name, 'id': str(uuid4())},
                page_content=html_content
            )
            documents.append(document_chunk)

    return documents, ids


In [9]:
ppc_2018_path = 'data/ppc_2018.pdf'
ppc_2018_documents, ppc_2018_ids = parse_document_to_documents(ppc_2018_path, 'ppc2018')

In [10]:
len(ppc_2018_documents)

405

In [12]:
from time import sleep

metadata_documents_2018 = ppc_2018_documents

for document in metadata_documents_2018:
    metadata = get_metadata(document.page_content, 'llama-3.1-70b-versatile')
    print(metadata)
    document.metadata.update(metadata.dict())
    sleep(7)

topico='Apresentação' palavras_chave='UFFS, Ministério da Educação, Pró-Reitoria de Graduação' possiveis_perguntas='Quando a Universidade Federal da Fronteira Sul foi criada?, Qual é a sede da Universidade Federal da Fronteira Sul?, Quais são os campi da Universidade Federal da Fronteira Sul?'
topico='Corpo docente e endereço da Reitoria' palavras_chave='Reitoria, Reitor, Vice-Reitor, Pró-Reitores' possiveis_perguntas='Quem é o Reitor da UFFS?, Quais são os nomes dos Pró-Reitores?'
topico='Corpo docente' palavras_chave='Dirigentes, Chapecó-SC, Diretora de Campus, Coordenadora Administrativa, Coordenador Acadêmico' possiveis_perguntas='Quem é o coordenador acadêmico de Chapecó-SC?, Quem é a diretora de Campus de Chapecó-SC?, Quem é a coordenadora administrativa de Chapecó-SC?'
topico='Corpo Docente do Campus de Cerro Largo' palavras_chave='corpo docente, dirigentes, Campus de Cerro Largo' possiveis_perguntas='Quais são os nomes dos dirigentes do Campus de Cerro Largo, Quem é o diretor d

In [15]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [16]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [18]:
import time

index_name = "index-ppc-markdown-ids-metadata"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [19]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [20]:
def add_documents_to_vector_store(vector_store, documents, uuids):
    """Add documents with unique IDs to the vector store."""
    print(vector_store.add_documents(documents=documents, ids=uuids))

In [21]:
add_documents_to_vector_store(vector_store, metadata_documents_2018, ppc_2018_ids)

['24392661-44c5-4486-90cc-2889e16f66b2', '406f3db9-f7e8-4871-9e05-1f6dda76f49a', '84f167e5-b2a1-448a-b5c1-c239f0107f3c', 'c51a54cd-a3e5-4219-b1e6-6001b5aa58f4', '20beb2de-1379-4e03-935e-b80fd85140d6', 'da9fa294-7650-4053-b882-01e7b017af37', '71f1617a-9ceb-4d43-a0a0-59a3336e4f42', '38f306f5-6d75-47de-9c9f-f1edb7d39029', '9cd56ff9-bcaf-42c4-baf1-e1d4aa332636', 'a67bc140-9c53-4a6a-8e3c-23454ab2c74d', 'd41be06e-457c-4e30-b200-130400042130', 'cfba8883-eae5-4c6a-91c8-043c023a70c4', 'b1cdc0c7-69b8-4ea2-8d14-c74d6681441f', '5bd6341b-49b0-4dfa-97c7-aee550773c35', '4dc2fcca-ab7c-4dd0-a4bc-7e4369982912', 'ff61dfdc-0c35-4e51-b59b-7a9134c7ea4d', '90526c93-6f75-481b-955a-cc3f15ad8cee', '04a4911b-4a5b-43e2-9ca2-bea3a7cce517', 'ec2d297f-6e4c-48b4-8afa-f969e48779b9', 'ec963cee-015f-4913-a65f-329c7b9b54bf', '113187f1-fb87-4403-ac8b-cfa0cf66d81a', '8c5a61d0-dcba-4a6c-ac60-2a0b740578de', '0065b426-f526-4d3c-b02e-551325135fc8', 'e8c7f5e6-da59-4e21-9c5b-08806c677dba', 'f9518bc2-d724-4259-9008-b09ae3b6d138',

In [55]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

# Definindo os campos de metadados disponíveis
metadata_field_info = [
    AttributeInfo(
        name="topico",
        description="Identifique do que se trata a pergunta: ementa de XXX, grade curricular , regulamento XXX, regras, laboratório, objetivos, infraestrutura, corpo docente, etc. Exemplos: ementa de Grafos, Instrução de validação de disciplinas, etc",
        type="string",
    ),
    AttributeInfo(
        name="palavras_chave",
        description="Lista de palavras-chave extraídas do chunk que representam os principais conceitos.",
        type="string",
    ),
    AttributeInfo(
        name="source",
        description="Fonte do documento. Opções são ppc2018 e pcc2024.",
        type="string",
    )
]

# Descrição do conteúdo do documento
document_content_description = "Pedaço de texto do Plano Pedágogico do Curso de cienca da computação"




In [57]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=4
)

# Definindo o LLM e o retriever
retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_store,
    document_content_description,
    metadata_field_info,
    search_kwargs={'k': 5}

)


In [59]:
context = retriever.invoke("Ementa de Computação Distribuída source:ppc_2018")
for ctx in context:
  print(ctx.metadata.get('topico'))

Grade curricular
Grade curricular
Grade Curricular de Ciência da Computação
Projeto Pedagógico Institucional (PPI) e organização curricular
Objetivos do Curso de Ciência da Computação


In [61]:
# context

## Não deu muito certo adicionar no metadata, vou adaptar para outro indice adicionando no content mesmo

In [65]:
from langchain_core.documents import Document

def adapt_metadata(document: Document) -> Document:
    metadata_extra = {key: value for key, value in document.metadata.items() if key not in ['id', 'source']}

    metadata_xml = "<metadata>\n"
    for key, value in metadata_extra.items():
        metadata_xml += f"{key}: {value},\n"
    metadata_xml = metadata_xml.rstrip(",\n")
    metadata_xml += "\n</metadata>\n\n"

    new_page_content = metadata_xml + document.page_content
    new_metadata = {key: value for key, value in document.metadata.items() if key in ['id', 'source']}

    return Document(page_content=new_page_content, metadata=new_metadata)

In [68]:
in_content_metadata_documents_2018 = [adapt_metadata(document) for document in metadata_documents_2018]

In [71]:
import time

index_name = "index-ppc-markdown-ids-metadata-in-content"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [72]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [74]:
add_documents_to_vector_store(vector_store, in_content_metadata_documents_2018, ppc_2018_ids)

['24392661-44c5-4486-90cc-2889e16f66b2', '406f3db9-f7e8-4871-9e05-1f6dda76f49a', '84f167e5-b2a1-448a-b5c1-c239f0107f3c', 'c51a54cd-a3e5-4219-b1e6-6001b5aa58f4', '20beb2de-1379-4e03-935e-b80fd85140d6', 'da9fa294-7650-4053-b882-01e7b017af37', '71f1617a-9ceb-4d43-a0a0-59a3336e4f42', '38f306f5-6d75-47de-9c9f-f1edb7d39029', '9cd56ff9-bcaf-42c4-baf1-e1d4aa332636', 'a67bc140-9c53-4a6a-8e3c-23454ab2c74d', 'd41be06e-457c-4e30-b200-130400042130', 'cfba8883-eae5-4c6a-91c8-043c023a70c4', 'b1cdc0c7-69b8-4ea2-8d14-c74d6681441f', '5bd6341b-49b0-4dfa-97c7-aee550773c35', '4dc2fcca-ab7c-4dd0-a4bc-7e4369982912', 'ff61dfdc-0c35-4e51-b59b-7a9134c7ea4d', '90526c93-6f75-481b-955a-cc3f15ad8cee', '04a4911b-4a5b-43e2-9ca2-bea3a7cce517', 'ec2d297f-6e4c-48b4-8afa-f969e48779b9', 'ec963cee-015f-4913-a65f-329c7b9b54bf', '113187f1-fb87-4403-ac8b-cfa0cf66d81a', '8c5a61d0-dcba-4a6c-ac60-2a0b740578de', '0065b426-f526-4d3c-b02e-551325135fc8', 'e8c7f5e6-da59-4e21-9c5b-08806c677dba', 'f9518bc2-d724-4259-9008-b09ae3b6d138',

In [86]:
relevant_docs = vector_store.similarity_search("Ementa de Cálculo II")
print(len(relevant_docs))
context = "\n".join([doc.page_content.split('</metadata>')[1] for doc in relevant_docs])
context

4


'\n\n| Código | COMPONENTE CURRICULAR | Créditos | Horas |\n|---|---|---|---|\n| Código | COMPONENTE CURRICULAR | Créditos | Horas |\n| GEX392 | CÁLCULO II | 4 | 60 |\n| EMENTA |   |   |   |\n| Algumas técnicas de integração e aplicações da integral. Funções de várias variáveis. Limi- te e continuidade de funções de várias variáveis. Derivadas parciais e aplicações. Gradiente. Diferenciabilidade. Multiplicadores de Lagrange. Integrais múltiplas. |   |   |   |\n| OBJETIVO |   |   |   |\n| Estudar mais algumas técnicas de integração de uma função de uma variável e aplicá-las na resolução de problemas. Introduzir as principais ferramentas do cálculo diferencial e inte- gral de funções de várias variáveis, abordando suas aplicações. |   |   |   |\n| REFERÊNCIAS BÁSICAS |   |   |   |\n| FLEMMING, D. M.; GONÇALVES, M. B. Cálculo A: funções, limite, derivação e inte- gração. 6. ed. São Paulo: Makron Books, 2007. ______. Cálculo B: funções de várias variáveis, integrais múltiplas, integrais cu